# Flood Insurance Data Cleaning
_Calvin Whealton_

This notebook processes the redacted National Flood Insurance Program redacted claims dataset. The data was obtained from https://www.fema.gov/media-library/assets/documents/180374. Data includes the claims for 1970 to 2019 and in addition to many characteristics of the type of claim it includes the zip code of the claim. The main values that will be analyzed for this work is the amount paid on the claims.

In [30]:
import os
import pandas as pd
import geopandas as gpd

In [3]:
# directory where the data is stored
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/FIMA_NFIP_Redacted_Claims_Data_Set')

# reading in file
claims = pd.read_csv('openFEMA_claims20190831.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,11,15,22,23,31,32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
claims.head()

,agriculturestructureindicator,asofdate,basefloodelevation,basementenclosurecrawlspacetype,reportedcity,condominiumindicator,policycount,countycode,crsdiscount,dateofloss,...,amountpaidonincreasedcostofcomplianceclaim,postfirmconstructionindicator,ratemethod,smallbusinessindicatorbuilding,state,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,yearofloss,reportedzipcode,primaryresidence
0,NaN,2019-08-31,NaN,0.0,OCEANSIDE,N,1.0,6073.0,0.00,1998-02-07,...,NaN,N,7,NaN,CA,200000.0,50000.0,1998,92056,NaN
1,NaN,2019-08-31,NaN,0.0,NEW ORLEANS,N,1.0,22071.0,0.00,2005-08-29,...,NaN,N,7,NaN,LA,100000.0,40000.0,2005,70131,Y
2,NaN,2019-08-31,NaN,0.0,NAVARRE,N,1.0,12113.0,0.05,1998-09-28,...,0.0,N,1,NaN,FL,100000.0,50000.0,1998,32566,NaN
3,NaN,2019-08-31,NaN,1.0,BEAUFORT,N,1.0,45013.0,0.00,1994-10-07,...,NaN,N,7,NaN,SC,100000.0,25000.0,1994,29902,NaN
4,NaN,2019-08-31,NaN,0.0,MELBOURNE,N,1.0,12009.0,0.00,1996-03-11,...,NaN,Y,7,NaN,FL,100000.0,25000.0,1996,32940,NaN


In [8]:
claims.shape

(2422370, 39)

In [9]:
min(claims['yearofloss']),max(claims['yearofloss'])

(1970, 2019)

In [10]:
len(claims['reportedzipcode'].unique())

43913

Focus is on housing. Small business, agricultural buildings, non-profit buildings, and places of worship are not of immediate interest.

In [11]:
claims.drop(claims[claims['houseworship']=='Y'].index,inplace=True)
claims.drop(claims[claims['agriculturestructureindicator']=='Y'].index,inplace=True)
claims.drop(claims[claims['nonprofitindicator']=='Y'].index,inplace=True)
claims.drop(claims[claims['smallbusinessindicatorbuilding']=='Y'].index,inplace=True)

In [12]:
claims.shape

(2422370, 39)

The metadata states that a negative claims amount means that the check was not cashed and had to be reissued. Therefore, the payment would still presumptively be positive. The claims are divided into building and contents. For the purpose of this analysis, both would be considered as representing a damage to the structure.

In [13]:
min(claims['amountpaidonbuildingclaim']),max(claims['amountpaidonbuildingclaim'])

(-29207.05, 10000000.0)

In [14]:
claims['amountpaidonbuildingclaim'] = claims['amountpaidonbuildingclaim'].abs()
claims['amountpaidoncontentsclaim'] = claims['amountpaidoncontentsclaim'].abs()
min(claims['amountpaidonbuildingclaim']),max(claims['amountpaidonbuildingclaim'])

(0.0, 10000000.0)

Making a column that will be used in aggregating the losses to the monthly values.

In [16]:
claims['yearmonthofloss'] = claims['dateofloss'].str[:-3]

In [17]:
claims.head()

,agriculturestructureindicator,asofdate,basefloodelevation,basementenclosurecrawlspacetype,reportedcity,condominiumindicator,policycount,countycode,crsdiscount,dateofloss,...,postfirmconstructionindicator,ratemethod,smallbusinessindicatorbuilding,state,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,yearofloss,reportedzipcode,primaryresidence,yearmonthofloss
0,NaN,2019-08-31,NaN,0.0,OCEANSIDE,N,1.0,6073.0,0.00,1998-02-07,...,N,7,NaN,CA,200000.0,50000.0,1998,92056,NaN,1998-02
1,NaN,2019-08-31,NaN,0.0,NEW ORLEANS,N,1.0,22071.0,0.00,2005-08-29,...,N,7,NaN,LA,100000.0,40000.0,2005,70131,Y,2005-08
2,NaN,2019-08-31,NaN,0.0,NAVARRE,N,1.0,12113.0,0.05,1998-09-28,...,N,1,NaN,FL,100000.0,50000.0,1998,32566,NaN,1998-09
3,NaN,2019-08-31,NaN,1.0,BEAUFORT,N,1.0,45013.0,0.00,1994-10-07,...,N,7,NaN,SC,100000.0,25000.0,1994,29902,NaN,1994-10
4,NaN,2019-08-31,NaN,0.0,MELBOURNE,N,1.0,12009.0,0.00,1996-03-11,...,Y,7,NaN,FL,100000.0,25000.0,1996,32940,NaN,1996-03


In [21]:
claims['GEOID10_str'] = claims['reportedzipcode'].apply(lambda x: '{0:0>5}'.format(x))

In [52]:
claims_for_groupby = claims.filter(['GEOID10_str','yearmonthofloss','amountpaidoncontentsclaim','amountpaidonbuildingclaim'])

Loading a zip code shapefile that will be used to evaluated valid zip codes. The valid zip codes are those in the US Census ZCTA (Zip Code Tabulation Area) shapefile. The shapefile has been clipped to the 48 contiguous states.

In [31]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/tdi_capstone/data/geo_data/tl_2019_us_zcta510_clipped48contig')
zip_shape = gpd.read_file('clipped48contig.shp')

In [56]:
valid_zips = zip_shape['ZCTA5CE10'].values

Looping through the dataframe and dropping rows (zip codes) that are not in the list of valid zip codes. Pre-processing the claims before using group by to reduce the number of results.

Using the `isin()` function because it is faster than looping through the dataframe.

In [59]:
claims_for_groupby = claims_for_groupby.loc[claims_for_groupby['GEOID10_str'].isin(valid_zips)]

In [60]:
claims_for_groupby.head()

,GEOID10_str,yearmonthofloss,amountpaidoncontentsclaim,amountpaidonbuildingclaim
42,22973,1995-06,0.0,0.00
43,22973,1996-09,0.0,0.00
44,29572,1999-09,0.0,0.00
45,33441,2005-10,0.0,0.00
46,32507,2004-09,60000.0,239509.58


In [61]:
claims_gb = claims_for_groupby.groupby(['GEOID10_str','yearmonthofloss']).sum()

In [62]:
claims_gb.head()

amountpaidoncontentsclaim  \
GEOID10_str yearmonthofloss                              
01001       1979-01                              86.50   
            1982-06                             375.00   
            1983-04                            1288.80   
            1984-05                           19723.85   
            1984-06                             110.00   

                             amountpaidonbuildingclaim  
GEOID10_str yearmonthofloss                             
01001       1979-01                             650.00  
            1982-06                               0.00  
            1983-04                               0.00  
            1984-05                           40669.73  
            1984-06                            9227.68

Checking that nothing was lost in the groupby() operation.

In [63]:
claims_gb['amountpaidoncontentsclaim'].sum()

11369096508.289999

In [65]:
claims_for_groupby['amountpaidoncontentsclaim'].sum()

11369096508.289995

Converting the different types of flood zones into binary variables (dummy variables). Result will be a 0 or 1 depending on the flood zone.

In [67]:
claims_gb['amountpaid'] = claims_gb['amountpaidoncontentsclaim'] + claims_gb['amountpaidonbuildingclaim']

In [70]:
min(sorted(claims_for_groupby['yearmonthofloss'])),max(sorted(claims_for_groupby['yearmonthofloss']))

('1973-04', '2019-08')

In [84]:
zips,dates = zip(*claims_gb.index)
zips2 = sorted(list(set(zips)))
dates2 = sorted(list(set(dates)))

In [85]:
claims_ts = pd.DataFrame({'GEOID10_str':zips2})

In [86]:
for d in dates2:
    claims_ts[d] = 0

In [87]:
for vals in range(len(zips)):
    claims_ts.loc[claims_ts['GEOID10_str']==zips[vals],dates[vals]] = claims_gb['amountpaid'].values[vals]

In [89]:
claims_ts.sum()

GEOID10_str             inf
1973-04        1.326700e+04
1973-08        0.000000e+00
1974-04        1.408000e+03
1974-05        1.000000e+04
                   ...     
2019-04        1.729401e+07
2019-05        1.341423e+08
2019-06        9.334140e+07
2019-07        3.261849e+07
2019-08        1.222505e+06
Length: 542, dtype: float64

Checking values above and below indicates that the sums match to within rounding. Values of claims were not lost or gained.

In [90]:
claims_for_groupby.groupby('yearmonthofloss').sum()

,amountpaidoncontentsclaim,amountpaidonbuildingclaim
yearmonthofloss,,
1973-04,3547.00,9.720000e+03
1973-08,0.00,0.000000e+00
1974-04,0.00,1.408000e+03
1974-05,0.00,1.000000e+04
1974-08,0.00,1.750000e+04
...,...,...
2019-04,1815850.00,1.547816e+07
2019-05,16728891.67,1.174135e+08
2019-06,12740182.55,8.060122e+07


In [142]:
# number of zeros (no claims)
num_zero_claims = sum((ts_claims == 0).astype(int).sum(axis=1))

In [141]:
num_possible_claims = (ts_claims.shape[1]-1)*ts_claims.shape[0]

In [143]:
num_claims = num_possible_claims - num_zero_claims
num_claims

116602

In [92]:
os.chdir('/Users/calvinwhealton/Documents/GitHub/floods_housing_zipcode/data/processed_data')
claims_ts.to_csv('ts_claims_month.csv')